In [2]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers import Input, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D
from tensorflow.keras.layers import AveragePooling2D, MaxPooling2D, Dropout, GlobalMaxPooling2D, GlobalAveragePooling2D 
from tensorflow.keras.utils import to_categorical
from keras.utils import np_utils, print_summary
import pandas as pd
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import ModelCheckpoint
import tensorflow.keras.backend as k

In [3]:
import pandas as pd
data = pd.read_csv('data.csv')
dataset=np.array(data)
np.random.shuffle(dataset)
X= dataset
Y= dataset
X= X[:, 0:1024]
Y= Y[:, 1024]
X_train= X[0:70000, :]
X_train= X_train/ 255.
X_test= X[70000:72001, :]
X_test= X_test/255.

Y= Y.reshape(Y.shape[0], 1) 
Y_train= Y[0:70000, :]
Y_train= Y_train.T
Y_test= Y[70000:72001, :]
Y_test= Y_test.T

In [4]:
print("Number of training examples ="+ str(X_train.shape[0]))
print("Number of test samples= "+ str(X_test.shape[0]))
print("X_train shape: " + str(X_train.shape))
print("Y_train shape: " + str(Y_train.shape))
print("X_test shape: " + str(X_test.shape))
print("Y_test shape: " + str(Y_test.shape))

Number of training examples =70000
Number of test samples= 2000
X_train shape: (70000, 1024)
Y_train shape: (1, 70000)
X_test shape: (2000, 1024)
Y_test shape: (1, 2000)


In [5]:
image_x = 32
image_y = 32

train_y = np_utils.to_categorical(Y_train)
test_y = np_utils.to_categorical(Y_test) 
train_y = train_y.reshape(train_y.shape[1], train_y.shape[2])
test_y = test_y.reshape(test_y.shape[1], test_y.shape[2])
X_train = X_train.reshape(X_train.shape[0], image_x, image_y, 1)
X_test = X_test.reshape(X_test.shape[0], image_x, image_y, 1)

In [6]:
print("X_train shape: " + str(X_train.shape))
print("Y_train shape: " + str(train_y.shape))

X_train shape: (70000, 32, 32, 1)
Y_train shape: (70000, 37)


In [7]:
#building a model
def keras_model(image_x, image_y):
    num_of_classes = 37
    model = Sequential()
    model.add(Conv2D(filters=32, kernel_size=(5,5), input_shape=(image_x,image_y, 1), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='same'))
    model.add(Conv2D(64, (5,5), activation='relu'))
    model.add(MaxPooling2D(pool_size=(5,5), strides=(5,5), padding='same'))
    model.add(Flatten())
    model.add(Dense(num_of_classes, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    filepath= "devanagari.h5"
    checkpoint1 = ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')
    callbacks_list= [checkpoint1]
    
    return model, callbacks_list

In [8]:
model, callbacks_list= keras_model(image_x, image_y)
model.fit(X_train, train_y, validation_data=(X_test, test_y), epochs=5, batch_size= 64, callbacks=callbacks_list)
scores= model.evaluate(X_test, test_y, verbose=0)
print("CNN Error: %.2f%%" % (100 - scores[1] * 100))
print_summary(model)
model.save('devanagari.h5')

Train on 70000 samples, validate on 2000 samples
Epoch 1/5
69952/70000 [============================>.] - ETA: 0s - loss: 0.7448 - accuracy: 0.7874
Epoch 00001: val_accuracy improved from -inf to 0.91250, saving model to devanagari.h5
70000/70000 [==============================] - 142s 2ms/sample - loss: 0.7445 - accuracy: 0.7874 - val_loss: 0.3155 - val_accuracy: 0.9125
Epoch 2/5
69952/70000 [============================>.] - ETA: 0s - loss: 0.2186 - accuracy: 0.9336
Epoch 00002: val_accuracy improved from 0.91250 to 0.94750, saving model to devanagari.h5
70000/70000 [==============================] - 102s 1ms/sample - loss: 0.2186 - accuracy: 0.9336 - val_loss: 0.1961 - val_accuracy: 0.9475
Epoch 3/5
69952/70000 [============================>.] - ETA: 0s - loss: 0.1414 - accuracy: 0.9569
Epoch 00003: val_accuracy improved from 0.94750 to 0.95650, saving model to devanagari.h5
70000/70000 [==============================] - 101s 1ms/sample - loss: 0.1413 - accuracy: 0.9569 - val_loss: 